# Spatial data
Plan
* Coordinates of Earth
* Showing data on a map
* Different kinds of spatial data
* Standard SQL spatial query operators
* Index for spatial queries


In [ ]:
!docker container ls -a

In [ ]:
%%bash
docker run \
--rm \
--name my_mysql \
-v $(pwd)/mysql_databasefiles:/var/lib/mysql \
-p 3306:3306 \
-e MYSQL_ROOT_PASSWORD=deterentysker!42snapsnap \
-d \
mysql
echo "MySQLRunning"

# Spatial data
![](https://cdn-images-1.medium.com/max/1200/1*e7w5S04WL7qICzeDNFP_bg.png)

# Spatial queries
* Distance between points
    * find points within a radius from an other point
    * find closes point
    * find the point which is closest to all other
* Three kind of fundamental shapes
    * points
    * lines (roads, rivers, hiking trails, tracks)
    * areas (countries, lakes, forests, parks,...)

# Spherical coordinates
![](https://cdn-images-1.medium.com/max/1600/0*fB5jHfIBZzI_TTGi.png)


# Spherical coordinate systems
* GPS (origin as in the previous slide - coordinates are angles in degrees - floats)
* This is a huge (and annoying) issue:
    * Where do we put origin
    * What do we measure
    * Distance between points are hard
    
Using GPS coordinates Copenhagen center is at lattitude, longitude: (55.6867243, 12.5700724) 


#  Coordinate Reference System Identifier or SRID
There is an organization named "Open Geospatial Consortium" which maintain a list of know coordinate systems.

They are available in MySQL:
```mysql
SELECT `srs_name`, `srs_id`
FROM INFORMATION_SCHEMA.ST_SPATIAL_REFERENCE_SYSTEMS
```

Three important SRID for MySQL:
* 0: The raw cartesian (X,Y) on a infinite flat map
* 4326: Standard Geographic Coordinate System (Geodata must be in this srid for MySQL)
* 25832: [Europe](https://epsg.io/25832) (Large and medium scale topographic mapping and engineering survey)

# Spatial sample data

Finding data:
We will look at some data for Copenhagen. The Copenhagen municipality has a lot of data at:
http://wfs-kbhkort.kk.dk/web/


Point data (where are there currently given permission to dig in the ground :-) ""):
http://wfs-kbhkort.kk.dk/k101/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=k101:erhv_gravetilladelser_aktiv&maxFeatures=50&outputFormat=application%2Fjson

Notice the `maxFeatures=50`in the link. To get the full dataset it is necessary to remove that:
http://wfs-kbhkort.kk.dk/k101/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=k101:erhv_gravetilladelser_aktiv&outputFormat=application%2Fjson




# Getting the data into the database
Three options (well, there are many, but lets look at two)
* geojson
* CSV
* Coded

### Geojson
* A standard from internet engineering task force (http://geojson.org).
* Wrt. MySQL - it is a two step procedure
    * Load the (geo)json into a table
    * Extract the relevant part of the json out into a materialized view (to make it possible to work with the locations)

## Your turn (10 min):

1. On the [spatial data of Copenhagen](http://wfs-kbhkort.kk.dk/web/;jsessionid=9C93A3CFBDBF78F5378C99FA64BE60B0?wicket:bookmarkablePage=:org.geoserver.web.demo.MapPreviewPage), find the category for bikeracks (cykelstativ).
2. Explore the geojson format.
3. In particular - notice the structure of the data, which is a mixture of geo and other info for each bikerack
4. Note the unusual (non-gps) coordinates. Find the part of the geojson which indicates the coordinate system used



### CSV
* Straight forward (well - sort of)
* Huge number of columns (30)
    * The mysql statement [LOAD DATA](https://dev.mysql.com/doc/refman/8.0/en/load-data.html) can do the job
    * LOAD DATA can ignore some columns

# Loading data - srid change
Cannot (to my knowledge) be done using the standard tools to load into MySQL. 

So we need to write a loader program to load the data and convert the locations.

# Code to create a "sql script"



# Coordinate convertion using pyproj

In [ ]:
from pyproj import Proj, transform
from re import sub

inProj = Proj(init='EPSG:25832')
outProj = Proj(init='epsg:4326')

def transformer(x,y):
    # point is on the form "722942.66 6173097.7"
    ll = transform(inProj, outProj, x,y)
    return str(ll[0])+' '+str(ll[1])

def geom_transformer(geom):
    reg_ex="([0-9.]+) ([0-9.]+)" # pick out two numbers with a space inbetween
    return sub(reg_ex, lambda m: transformer(m.group(1), m.group(2)), geom)

geom_transformer("POINT (722942.66 6173097.7)")

# Converting to sql

* Write a program that read the input format, and inserts data into the database
* Write a program that read the input format, and creates a "dump" file (creates tables and insert statements).

### I will do the latter - create a reader which creates sql output

# Digging permits in Copenhagen
(It will allow us to see where they are working on Fibernet in Copenhagen)

http://wfs-kbhkort.kk.dk/k101/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=k101:erhv_gravetilladelser_aktiv&outputFormat=csv


Examine the csv format file for which data to include in the database.

<sub><sub>Note to Kasper - open the file and look</sub></sub>

## Overall function

In [ ]:
# import data as: wfs-kbhkort.kk.dk/k101/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=k101:erhv_gravetilladelser_aktiv&outputFormat=csv
import os
import csv

our_headers = {'sagsnr':'INT', 
               'grave_art':'VARCHAR(100)', 
               'bygherre':'VARCHAR(100)', 
               'wkb_geometry':'GEOMETRY NOT NULL SRID 4326'}

def makeSQLFile(file_name, headers=our_headers):
    cwd = os.getcwd() # current working directory
    inputfile = open(f"{cwd}/{file_name}.csv","r")
    outputfile = open(f"{cwd}/{file_name}.sql", "w+")
    table = createTable(file_name, headers)
    inserts = makeInsertStatements(file_name, headers, inputfile, outputfile)
    outputfile.write(table)
    outputfile.write(inserts)
    outputfile.close()
    
our_headers

In [ ]:
def createTable(name, headers):
    sql = f"drop table if exists {name};\n"
    sql += f"create table {name} (\n"
    sql += ", \n".join([f"\t{header} {sql_type}" for header,sql_type in headers.items()])
    sql += f",\n\tprimary key({list(headers.keys())[0]})\n);\n"
    print(f'created table {name}')
    return sql
print(createTable('digging', our_headers))

# Making the insert statements
The format of an insert statement is:

```mysql
INSERT INTO table_name ( field1, field2,...fieldN )
   VALUES
   ( value1, value2,...valueN );
```

A minor issue is we read all values from the CSV file as text, but need to convert them to their sql type.

In [ ]:
def valueOf(v, sql_type):
    if sql_type == "INT":
        return v
    if sql_type[0:8] == "GEOMETRY":
        c = geom_transformer(v) # This is where I convert from EPSG:25832' to 'epsg:4326'
        return f'ST_GeomFromText("{c}", 4326)'
    return f'"{v}"'
print( valueOf(77,'INT'))
print( valueOf("POINT (722942.66 6173097.7)", 'GEOMETRY NOT NULL SRID 4326') )
print( valueOf('Fibernet', 'VARCHAR(100)'))

# Making insert statements


In [ ]:
def makeInsertStatements(name, headers, infile, outfile):
    print("Writing inserts ",end='')
    headerline = infile.readline()
    csv_headers = headerline.rstrip().split(",")
    headerIndex = { h : csv_headers.index(h) for h in headers.keys()}
    sql = ""
    csv_in = csv.reader(infile, delimiter=',', quotechar='"')
    for row in csv_in:
        print('.',end='')
        if row[-1] != "": # some positions are missing
            sql_values = [valueOf(row[headerIndex[k]], headers[k]) for k in headers.keys()]
            values_combined = ','.join(sql_values)
            columns = ','.join(headers.keys())
            sql += f"INSERT INTO {name} ( {columns} ) VALUES ({values_combined});\n"
    return sql

In [ ]:
# makeSQLFile('digging')

# Your turn
1. The generated datafile is in the 'data' folder at https://github.com/datsoftlyngby/soft2019spring-databases
2. Make a schema (I called mine copenhagen) for the table, and load the table from the 'digging.sql'
3. Which kind of activities are they digging for?
4. How many diggings are taking place within a radius of 500 meters from the main square (Rådhuspladsen) 
    * main square is at (55.6867243, 12.5700724)

In [ ]:
import sys
import mysql.connector

def rootconnect():
    try:
        pw = 'deterentysker!42snapsnap'
        conn = mysql.connector.connect( host='localhost', database='copenhagen',user='root', password=pw)
        conn.autocommit = True
        return conn;
    except Exception as ex:
        print(str(ex), file=sys.stderr)
    

conn = rootconnect()

def sqlQuery(sqlString):
    global conn
    try:
        if not conn.is_connected():
            conn = rootconnect()
        cursor = conn.cursor()
        cursor.execute(sqlString)
        res = cursor.fetchall()
        return res
    except Exception as ex:
        print(str(ex), file=sys.stderr)
    finally:    
        cursor.close()

# Showing things on a map (from jupyter)
### Plotting to Maps with Folium

https://github.com/python-visualization/folium

`pip install folium`

### [Starter examples](https://python-visualization.github.io/folium/quickstart.html)


In [ ]:
fibernet = sqlQuery("""
with dig as (
        select grave_art as kind,
            bygherre as constructor,
            ST_Longitude(wkb_geometry) as lng,
            ST_Latitude(wkb_geometry) as lat
        from copenhagen.digging)
select * 
from dig 
where kind = "Fibernet"
""")
fibernet

In [ ]:
import folium
# create empty map zoomed in on Copenhagen
cph_loc = (55.6867243, 12.5700724)
map = folium.Map(location=cph_loc, zoom_start=12)

for each in fibernet:
    folium.Marker([each[2], each[3]], popup=each[1]).add_to(map)
  
map

# Geometric datatypes
A Geometric value has several aspects:
* Its type (point, line, area)
* SRID (0 is the flat plane, 4326 is the normal latitude, longitude coordinates
* Its coordinates
* interior, boundary, and exterior
* MBR (minimum bounding rectangle)
* closed or not closed
* dimension. A geometry can have a dimension of −1, 0, 1, or 2 (empty, point, line, area)

# Spatial operators

## There are many!!
See section [12.16.1 Spatial Function Reference](https://dev.mysql.com/doc/refman/8.0/en/spatial-function-reference.html)

#### In particular
* MBRContains(), MBRCoveredBy(), MBRCovers(), MBRDisjoint(), MBRIntersects(), MBROverlaps(), MBRTouches(), MBRWithin()
* ST_Crosses(), ST_Contains(), ST_Touches(), ...
* ST_Distance(), ST_Distance_Sphere()

# Are they digging in any of the parks of Copenhagen

http://wfs-kbhkort.kk.dk/ $\rightarrow$ Layer Preview $\rightarrow$ parkregister

http://wfs-kbhkort.kk.dk/k101/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=k101:parkregister&maxFeatures=50&outputFormat=application%2Fjson

(Btw, I handedit the link above - I start out from picking "geojson", and then remove "maxFeatures=50", and change output format to csv

http://wfs-kbhkort.kk.dk/k101/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=k101:parkregister&outputFormat=csv

In [ ]:
park_headers = {
    'areal_id':'INT',
    'navn_arealer': 'VARCHAR(100)',
    'parktype' : 'VARCHAR(100)',
    'wkb_geometry' : 'GEOMETRY NOT NULL SRID 4326'
}
# makeSQLFile('parkregister', park_headers)

# Indexing spatial data
![](images/R-tree.png)

# R-index for geospatial data
Walnut Creek, California (https://geoffboeing.com/2016/10/r-tree-spatial-index-python/)

| area | area and points |
|------|-----|
|![](https://i2.wp.com/geoffboeing.com/wp-content/uploads/2016/10/walnut-creek-city-boundary.png)|![](https://i0.wp.com/geoffboeing.com/wp-content/uploads/2016/10/walnut-creek-boundary-intersections.png)|


| area | area and points |
|------|-----|
|![](https://i0.wp.com/geoffboeing.com/wp-content/uploads/2016/10/los-angeles-boundary-subdivided.png)|![](https://i2.wp.com/geoffboeing.com/wp-content/uploads/2016/10/los-angeles-boundary-quadrats-intersections.png)|

# Sample
Are they digging in any parks?
```mysql
select 
    navn_arealer, 
    parktype,
	grave_art, 
    ST_Longitude(digging.wkb_geometry) as lng,
	ST_Latitude(digging.wkb_geometry) as lat
from digging, parkregister
where ST_Contains(parkregister.wkb_geometry, digging.wkb_geometry);
```

In [ ]:
park_digging = sqlQuery("""
select navn_arealer, parktype,
    grave_art, 
    ST_Longitude(digging.wkb_geometry) as lng,
    ST_Latitude(digging.wkb_geometry) as lat
from digging,parkregister
where ST_Contains(parkregister.wkb_geometry, digging.wkb_geometry);
""")

import folium
# create empty map zoomed in on Copenhagen
cph_loc = (55.6867243, 12.5700724)
map = folium.Map(location=cph_loc, zoom_start=12)

for each in park_digging:
    folium.Marker([each[3], each[4]], popup=each[2]).add_to(map)

In [ ]:
  map

# Adding indexes
```mysql
ALTER TABLE digging ADD SPATIAL INDEX(wkb_geometry);
ALTER TABLE parkregister ADD SPATIAL INDEX(wkb_geometry);
```

or dropping it:
```mysql
DROP INDEX wkb_geometry ON digging;
DROP INDEX wkb_geometry ON parkregister;
```



# Assignment

At https://github.com/datsoftlyngby/soft2019spring-databases/blob/master/assignments/assignment10.md